In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_timestamp
from configparser import ConfigParser
import pydeequ
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.analyzers import *

In [14]:
Parser = ConfigParser()
configFile = r"..\config\checks.json"
Parser.read(configFile)

['..\\config\\checks.json']

In [15]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "C:\Program Files (x86)\PostgreSQL\pgJDBC\postgresql-42.2.18.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [16]:
def get_datasets_verify_constrains(sections):
    if Parser.get(sections,"file_format")=="csv":
        tables = (spark.read.format('csv').options(header=True,inferSchema=True).load(Parser.get(sections, 'source')))
    elif Parser.get(sections,"file_format")=="parquet":
        tables = (spark.read.parquet(Parser.get(sections,'source')))
    
    #tables = (spark.read.format(Parser.get(sections,'file_format')).options(header=True,inferSchema=True).load(Parser.get(sections,'source')))
    print(sections)
    #tables.show()
    checker = Parser.get(sections,'checks').split(',')
    checker.insert(0,"check")
    checks =".".join(checker)
    check = Check(spark, CheckLevel.Error, Parser.get(sections,'check_name'))
    checkResult = (VerificationSuite(spark)
                   .onData(tables)
                   .addCheck(eval(checks))
                   .run())
    checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
    checkResult_df_result = (checkResult_df.withColumn('dataset_name',lit(sections))
                         .withColumn('check_run_tsp',lit(current_timestamp())))
    checkResult_df_result.show()
    checks_result_path = str(Parser.get(sections, 'checks_result')) + sections
    checkResult_df_result.coalesce(1).write.csv(f'{checks_result_path}',header=True,mode='overwrite')


In [17]:
Parser.get("merchants", 'check_name')

'Sellbrite checks'

In [18]:
list(map(get_datasets_verify_constrains,Parser.sections()))

merchants
+----------------+-----------+------------+--------------------+-----------------+------------------+------------+--------------------+
|           check|check_level|check_status|          constraint|constraint_status|constraint_message|dataset_name|       check_run_tsp|
+----------------+-----------+------------+--------------------+-----------------+------------------+------------+--------------------+
|Sellbrite checks|      Error|     Success|CompletenessConst...|          Success|                  |   merchants|2020-12-30 20:10:...|
|Sellbrite checks|      Error|     Success|UniquenessConstra...|          Success|                  |   merchants|2020-12-30 20:10:...|
|Sellbrite checks|      Error|     Success|ComplianceConstra...|          Success|                  |   merchants|2020-12-30 20:10:...|
+----------------+-----------+------------+--------------------+-----------------+------------------+------------+--------------------+

users
+----------------+-----------+-

[None, None, None]